# Text model explanation using Anchor Explanations

### Imports and installs

In [1]:
import os
import random
import spacy

import numpy                 as np
import tensorflow            as tf
import tensorflow_hub        as hub
import tensorflow_datasets   as tfds

In [2]:
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
!pip install spacy lime pydictionary
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [4]:
os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "COMPRESSED"
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.3.0
Eager mode:  True
Hub version:  0.10.0
GPU is available


In [5]:
!rm -rf explainable_ai
!git clone https://github.com/kartikparnami/explainable_ai.git
from explainable_ai.anchor.anchor_text import AnchorText

Cloning into 'explainable_ai'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 18 (delta 0), reused 18 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Construct model and utilities

In [6]:
def mask_to_categorical(data, mask):
    mask = tf.one_hot(tf.cast(mask, tf.int32), 2)
    mask = tf.cast(mask, tf.float32)
    return data, mask

# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="yelp_polarity_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

train_examples_batch, train_labels_batch = next(iter(train_data.batch(10).map(mask_to_categorical)))

In [7]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = model.fit(train_data.shuffle(10000).batch(512).map(mask_to_categorical),
                    epochs=1,
                    validation_data=validation_data.batch(512).map(mask_to_categorical),
                    verbose=1)

657/657 [==============================] - 29s 44ms/step - loss: 0.2364 - accuracy: 0.9000 - val_loss: 0.1936 - val_accuracy: 0.9210


In [8]:
def predict_lr_anchor(texts):
    preds = model.predict(texts)
    return np.array([0 if p[0] > p[1] else 1 for p in preds])

### Anchor text explanation and visualization

In [9]:
a_text = AnchorText()

#### Positive prediction example

In [10]:
text = "Cheap and delicious! I eat here about once a week because it fills me up for cheap price and the people who work there are really fun. it\'s like a subway sandwich restaurant for Mexican food. I love the burrito w/ Cheese, Beans & rice w/ sour cream, less than $5 and best in town for that price."
exp = a_text.explain_instance(text, predict_lr_anchor)
exp.visualize_results(text)

Anchor: price


Positive explanation and words that contribute the most towards the prediction. Anchor for the complete explanation is the word "price".

#### Negative prediction example

In [11]:
text = "Perhaps it was an off night, but we were really disappointed in our take-out. My husband's pork fried rice was not fried rice...still don't know what it was. His Gen. Tso's chicken was flat and my cashew chicken was flavorless. In the past it has been good food."
exp = a_text.explain_instance(text, predict_lr_anchor)
exp.visualize_results(text)

Anchor: flavorless


Negative explanation and words that contribute the most towards the prediction. Anchor for the complete explanation is the word "flavorless"